# QSimBench - Demo

This demo walks you through using **QSimBench** to explore available quantum benchmark data, sample reproducible outcomes, and access circuit metadata — all in a few lines of code.

First, install QSimBench from PyPI:

In [1]:
# %pip install qsimbench

Start your Python session and import the main functions:

In [2]:
from qsimbench import get_index, get_outcomes, get_metadata

Now let’s see which algorithms, sizes, and backends are available:

In [3]:
index = get_index()
for algorithm, sizes in index.items():
    print(f"Algorithm: {algorithm}")
    for size, backends in sizes.items():
        print(f"  Size: {size} → Backends: {backends}")
    print()

Algorithm: dj
  Size: 10 → Backends: ['aer_simulator', 'fake_fez', 'fake_kyiv', 'fake_marrakesh', 'fake_sherbrooke', 'fake_torino']
  Size: 11 → Backends: ['aer_simulator', 'fake_fez', 'fake_kyiv', 'fake_marrakesh', 'fake_sherbrooke', 'fake_torino']
  Size: 12 → Backends: ['aer_simulator', 'fake_fez', 'fake_kyiv', 'fake_marrakesh', 'fake_sherbrooke', 'fake_torino']
  Size: 13 → Backends: ['aer_simulator', 'fake_fez', 'fake_kyiv', 'fake_marrakesh', 'fake_sherbrooke', 'fake_torino']
  Size: 14 → Backends: ['aer_simulator', 'fake_fez', 'fake_kyiv', 'fake_marrakesh', 'fake_sherbrooke', 'fake_torino']
  Size: 15 → Backends: ['aer_simulator', 'fake_fez', 'fake_kyiv', 'fake_marrakesh', 'fake_sherbrooke', 'fake_torino']
  Size: 4 → Backends: ['aer_simulator', 'fake_fez', 'fake_kyiv', 'fake_marrakesh', 'fake_sherbrooke', 'fake_torino']
  Size: 5 → Backends: ['aer_simulator', 'fake_fez', 'fake_kyiv', 'fake_marrakesh', 'fake_sherbrooke', 'fake_torino']
  Size: 6 → Backends: ['aer_simulator', 'fak

Suppose you want to simulate 1024 shots of a QAOA circuit with 8 qubits on the aer_simulator backend.
You can do this with one function call:

In [4]:
counts = get_outcomes(
    algorithm="qaoa",
    size=8,
    backend="aer_simulator",
    shots=1024,
    seed=42                  # For reproducibility
)

print("Sampled outcome counts:", counts)

Sampled outcome counts: {'01101001': 32, '00011011': 164, '01101110': 13, '01001011': 15, '10110001': 44, '11100100': 209, '01010110': 41, '01101010': 23, '01011010': 7, '01001010': 3, '11010110': 12, '01100110': 7, '01001110': 39, '10010110': 34, '11101001': 11, '10100101': 11, '11000110': 13, '10010101': 19, '00010110': 15, '11010100': 7, '01011110': 11, '00101011': 8, '00111011': 8, '10101001': 30, '01001101': 1, '10111001': 18, '01101011': 6, '11000100': 10, '10010001': 12, '01101000': 11, '10011001': 11, '11101000': 3, '11011011': 2, '01000110': 15, '10010111': 19, '10010100': 13, '01001100': 1, '00011000': 2, '10100001': 9, '01011011': 1, '10110100': 12, '11101100': 2, '00111100': 1, '00101001': 11, '00010011': 1, '00110111': 2, '00011110': 10, '01101100': 11, '00111001': 4, '10010011': 7, '11100001': 6, '10101110': 2, '10110101': 11, '11100101': 3, '00011001': 5, '01100100': 2, '00001111': 2, '11110100': 1, '00101111': 1, '10011011': 2, '00011111': 1, '10100100': 2, '00011010': 

To analyze or reproduce an experiment, you often need the underlying circuit and configuration.
Fetch this info with:

In [5]:
meta = get_metadata("qaoa", 8, "aer_simulator")[0]["metadata"]

# Display the OpenQASM circuit, noise model, and backend config
print(f"Circuit:\n{meta['circuit']['circuit'][:300]}...")  # print first 300 chars
print()
print("Backed Config:\n", meta["backend"])

Circuit:
OPENQASM 2.0;
include "qelib1.inc";
gate gate_PauliEvolution(param0) q0,q1,q2,q3,q4,q5,q6,q7 { rzz(-5.627177312657698) q0,q2; rzz(-5.627177312657698) q0,q6; rzz(-5.627177312657698) q1,q5; rzz(-5.627177312657698) q1,q7; rzz(-5.627177312657698) q2,q3; rzz(-5.627177312657698) q3,q7; rzz(-5.627177312657...

Backed Config:
 {'name': 'aer_simulator', 'configuration': {'backend_name': 'aer_simulator', 'backend_version': '0.17.1', 'n_qubits': 35, 'basis_gates': ['ccx', 'ccz', 'cp', 'crx', 'cry', 'crz', 'cswap', 'csx', 'cu', 'cu1', 'cu2', 'cu3', 'cx', 'cy', 'cz', 'diagonal', 'ecr', 'h', 'id', 'mcp', 'mcphase', 'mcr', 'mcrx', 'mcry', 'mcrz', 'mcswap', 'mcsx', 'mcu', 'mcu1', 'mcu2', 'mcu3', 'mcx', 'mcx_gray', 'mcy', 'mcz', 'multiplexer', 'p', 'pauli', 'r', 'roerror', 'rx', 'rxx', 'ry', 'ryy', 'rz', 'rzx', 'rzz', 's', 'sdg', 'store', 'swap', 'sx', 'sxdg', 't', 'tdg', 'u', 'u1', 'u2', 'u3', 'unitary', 'x', 'y', 'z', 'break_loop', 'continue_loop', 'delay', 'for_loop', 'if_else', 'initialize

QSimBench supports two sampling strategies:
- sequential: Mimics pulling the next batch of real shots (good for streaming/online experiments)
- random: Draws batches at random from the dataset (good for statistical studies)

Try both and compare:

In [6]:
out_seq = get_outcomes("qaoa", 8, "aer_simulator", strategy="sequential", shots=500, seed=42)
out_rand = get_outcomes("qaoa", 8, "aer_simulator", strategy="random", shots=500, seed=42)

print("Sequential sample:", out_seq)
print("Random sample:", out_rand)

Sequential sample: {'00010110': 8, '00011010': 2, '00011011': 94, '00101011': 4, '01000110': 5, '01001011': 6, '01001110': 14, '01010110': 13, '01011110': 4, '01101000': 12, '01101001': 9, '01101010': 14, '10010001': 6, '10010011': 4, '10010100': 8, '10010101': 16, '10010110': 13, '10100101': 3, '10101001': 14, '11011011': 1, '11100100': 99, '11101001': 10, '00010111': 3, '00101001': 10, '00111001': 5, '01011010': 3, '01101011': 9, '10011001': 7, '10100001': 11, '10110001': 13, '10110101': 8, '10111001': 6, '11000110': 7, '11010110': 4, '11100001': 3, '01100100': 2, '01100110': 2, '01101100': 3, '10010111': 4, '10110100': 4, '11000100': 1, '11101000': 4, '00011110': 6, '10010010': 1, '11010100': 3, '00011001': 1, '00100100': 2, '00101000': 2, '00111011': 1, '01100000': 1, '01101110': 6, '11001000': 1, '00010011': 2, '01000111': 1, '11100110': 1, '01001010': 2, '01100011': 1, '10011100': 1}
Random sample: {'00000011': 1, '00010111': 7, '00011001': 2, '00011011': 87, '00011110': 2, '0011

By default, QSimBench caches dataset files to avoid repeated downloads.
To force a fresh download (if the data has been updated), use force=True:

In [7]:
counts = get_outcomes("QAOA", 8, "aer_simulator", shots=128, force=True)
counts

{'11100100': 17,
 '10110001': 5,
 '11101001': 5,
 '10100101': 1,
 '11010100': 2,
 '01001110': 7,
 '10110101': 3,
 '10010110': 4,
 '10111001': 3,
 '01001100': 1,
 '10101001': 6,
 '01011110': 3,
 '00011011': 23,
 '01101110': 3,
 '10010001': 3,
 '00010110': 4,
 '11000110': 2,
 '01101000': 3,
 '01010110': 3,
 '10010101': 1,
 '00010011': 2,
 '00011001': 3,
 '01001011': 7,
 '01100110': 2,
 '00101011': 2,
 '11101000': 2,
 '10011011': 1,
 '00001111': 1,
 '01101010': 3,
 '11100111': 2,
 '11010110': 1,
 '01101001': 1,
 '00111001': 1,
 '01000110': 1}